In [1]:
import pandas
import numpy as np

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

# Препроцессинг

Изначальный датасет очень большой и некоторые операции не возможны на моем ноутбуке в связи с нехваткой оперативной памяти, поэтому поделим датасет на `n` маленьких

In [4]:
data = pandas.read_csv("train.csv")
data.head()

,id3,user_id,id2,date,id1
0,714,464300,34,1,4
1,714,915655,34,1,4
2,316,262696,42,1,2
3,52,354280,4,1,10
4,581,218912,14,1,10


In [4]:
n_batches = 5
users = data["user_id"].unique()
users_batches = (users[i::n_batches] for i in range(n_batches))

for i, batch in enumerate(users_batches):
    i+=1
    print("batch %s from %s" % (i, n_batches))
    data[data["user_id"].isin(batch)].to_csv("data/raw/data_%s.csv" % i, index=False)

batch 1 from 5
batch 2 from 5
batch 3 from 5
batch 4 from 5
batch 5 from 5


# Пейплайн предобработки датасета

### Подготовка и обучение пейплайна

Считаем один блок данных, и протестируем на нем пейплайн.

In [38]:
dataset_name = "data/raw/data_1.csv"
data = pandas.read_csv(dataset_name)
data.head()

,id3,user_id,id2,date,id1
0,714,464300,34,1,4
1,590,1029729,63,1,9
2,703,861305,16,1,9
3,803,733016,40,1,7
4,219,448027,1,1,6


### OneHotEncoder
Закодируем все просмотры пользователя в некоторый вектор интересов, для этого обучим OHE на категориях и подкатегориях товаров.

In [70]:
ohe = OneHotEncoder()

ohe.fit(data[["id1","id2"]])

id1_columns = ["id1_%s" % i for i in range(data["id1"].unique().shape[0])]
id2_columns = ["id2_%s" % i for i in range(data["id2"].unique().shape[0])] 

x_column_names = ["user_id"] + id1_columns + id2_columns
column_names = x_column_names

### Векторизация просмотров пользователя

Это функция для получения веткора интересов пользователя.
По своей сути вектор отображает вероятность, что человек заинтересуется определенной темой, а так же вероятность того, что он заинтересуется подтемой определенной темы

In [161]:
def vectorize_user(log):
    # Преобразуем все id в one hot
    values = ohe.transform(log[["id1", "id2"]])
    
    #суммируем все вектора и получаем вектор истории просмотров
    values = np.array(values.sum(axis=0))[0].astype("int")
    
    user_id = np.array([data["user_id"].get_values()[0]])
    vector = np.concatenate([user_id, values])
    
    user_vector = pandas.DataFrame(columns=x_column_names, data=[vector])
    
    # нормализация вектора, для пролучения вероятностей
    for columns in [id1_columns, id2_columns]:
        vec_len = user_vector[columns].sum(axis=1)[0]
        
        for c in columns:
            user_vector[c] = user_vector[c]/vec_len
        
    return user_vector
    

Пример векторизации интересов пользователя:

In [163]:
data_ = data[data["user_id"] == 464300]
vectorize_user(data_)

,user_id,id1_0,id1_1,id1_2,id1_3,id1_4,id1_5,id1_6,id1_7,id1_8,...,id2_112,id2_113,id2_114,id2_115,id2_116,id2_117,id2_118,id2_119,id2_120,id2_121
0,464300,0.0,0.010563,0.193662,0.035211,0.105634,0.003521,0.021127,0.03169,0.267606,...,0.0,0.0,0.0,0.0,0.007042,0.024648,0.003521,0.0,0.056338,0.0


In [61]:
s = data_["user_id"]

In [69]:
s.get_values()[0:1]

array([147637])

In [200]:
train_data.to_csv("train_data.csv")

In [22]:
def process_ids(data):
    values = ohe.transform(data[["id1", "id2"]])
    values = np.array(values.sum(axis=0))[0].astype("int")
    
    user_id = np.array([data["user_id"].get_values()[0]])
    values = np.concatenate([user_id, values])
    
    return values

def process_y_labels(data):
    y = data["id3"].value_counts()[:5].tolist()
    y += [None] * (5-len(y))
    y = np.array(y)
    
    return y
    
def preprocss_user_data(data):
    values = process_ids(data)
    values = np.concatenate([values, process_y_labels(data)])
    
    values = values.reshape([1, values.shape[0]])
    row = pandas.DataFrame(data=values, columns=col_names)
    
    return row

In [15]:
len(col_names)

NameError: name 'col_names' is not defined

In [16]:
process_ids(data_)

NameError: name 'ohe' is not defined

In [141]:
def preprocss_data(data):
    result_data = pandas.DataFrame(columns=col_names)
    users = data["user_id"].unique()
    
    for user in log_progress(users):
        row = preprocss_user_data(data[data["user_id"] == user])
        result_data = pandas.concat([result_data, row])
    
    return result_data

In [153]:
train_dataset = preprocss_data(data[:100000])

A Jupyter Widget

In [193]:
train_dataset.to_csv("raw_train.csv")

In [194]:
features = col_names[:]
features = features[:-5]

In [197]:
train_data = pandas.DataFrame(columns=features+["y"])

for col in ["id3_1", "id3_2", "id3_3", "id3_4", "id3_1"]:
    d = train_dataset[features]
    d["y"] = train_dataset[col]
    
    train_data = pandas.concat([train_data, d])

/home/alxmamaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [199]:
train_data.head()

,user_id,id1_0,id1_1,id1_2,id1_3,id1_4,id1_5,id1_6,id1_7,id1_8,...,id2_113,id2_114,id2_115,id2_116,id2_117,id2_118,id2_119,id2_120,id2_121,y
0,464300,0,0,1,2,12,1,3,8,17,...,0,0,0,0,0,1,0,0,0,2
0,1029729,1,0,2,4,33,5,28,47,19,...,0,0,0,0,2,4,0,0,0,1
0,861305,1,0,2,3,28,4,30,39,17,...,0,0,0,0,3,3,0,0,0,2
0,733016,0,2,2,2,17,5,38,89,24,...,0,0,0,2,1,7,1,0,0,2
0,448027,0,0,0,0,11,1,6,11,3,...,0,0,0,0,1,1,0,0,0,1
